In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import json
import osmnx as ox

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

/home/cseadmin/dz/anaconda3/envs/dz/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


'/home/cseadmin/dz/TrafficFlowModel/data_process/gis_pipeline'

In [2]:
DATA_PATH = "../../data/"
TAXI_DATA_PATH = "../../data/taxi_after_proc/clean202006"
DATASET = "sz_taxi_202006"

MIN_LAT = 22.5311
MAX_LAT = 22.5517
MIN_LNG = 114.0439
MAX_LNG = 114.0633

START_DAY = 1
END_DAY = 30

DOWNSAMPLING_INTERVAL = 30
TRAJ_SPLIT_INTERVAL = 600
FLOW_AGG_INTERVAL_MINUTE = 15

def contains(lat, lng):
    return lat >= MIN_LAT and lat <= MAX_LAT and lng >= MIN_LNG and lng <= MAX_LNG

In [4]:
gps_file = open(os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "gps.csv"), "w")
trash = gps_file.write("id;x;y;time\n")

timedelta_downsampling = pd.Timedelta(seconds=DOWNSAMPLING_INTERVAL)
timedelta_traj_split = pd.Timedelta(seconds=TRAJ_SPLIT_INTERVAL)

traj_counter = 0
for taxi_file in tqdm(sorted(os.listdir(TAXI_DATA_PATH))):
    date = int(taxi_file.split("_")[0].split("-")[1])
    if date < START_DAY or date > END_DAY:
        continue
    if os.path.getsize(os.path.join(TAXI_DATA_PATH, taxi_file)) < 100:
        continue
    # df_taxi = pd.read_csv(os.path.join(TAXI_DATA_PATH, taxi_file), parse_dates=["gps_time"])
    df_taxi = pd.read_pickle(os.path.join(TAXI_DATA_PATH, taxi_file))
    if df_taxi.empty:
        continue

    line_buffer = []
    last_time = df_taxi.iloc[0]["gps_time"] + pd.Timedelta(seconds=-TRAJ_SPLIT_INTERVAL)
    for row in df_taxi.itertuples():
        if not contains(row[1], row[2]):
            continue
        if row[3] - last_time < timedelta_downsampling:  # resample: drop <30s
            continue
        if row[3] - last_time > timedelta_traj_split:
            if len(line_buffer) > 1:  # only store length>1 traj
                trash = gps_file.write("".join(line_buffer))
                traj_counter += 1
            line_buffer = []

        last_time = row[3]

        line_buffer.append(f"{traj_counter};{row[2]};{row[1]};{row[3]}\n")

gps_file.close()

  1%|          | 7060/617790 [02:22<3:26:08, 49.38it/s]


KeyboardInterrupt: 

In [3]:
# https://github.com/cyang-kth/fmm/issues/166
# https://github.com/cyang-kth/fmm/blob/master/example/osmnx_example/README.md

os.system(
    "ubodt_gen --network {} --network_id fid --source u --target v --output {} --delta 0.03 --use_omp"
    .format(
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "edges.shp"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "ubodt.txt")))
os.system(
    "fmm --ubodt {} --network {} --network_id fid --source u --target v --gps {} --gps_point -k 8 -r 0.003 -e 0.0005 --output {} --use_omp --output_fields id,opath,cpath,mgeom > {} 2>&1"
    .format(
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "ubodt.txt"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "edges.shp"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "gps.csv"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "mr.txt"),
        os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "fmm.log")))

[info][ubodt_gen_app_config.cpp:42 ] Start reading ubodt configuration from arguments
[info][ubodt_gen_app_config.cpp:70 ] Finish with reading ubodt arg configuration
[info][ubodt_gen_app_config.cpp:74 ] ----    Print configuration   ----
[info][network_config.cpp:6  ] NetworkConfig
[info][network_config.cpp:7  ] File name: ../data/fmm_data/edges.shp 
[info][network_config.cpp:8  ] ID name: fid 
[info][network_config.cpp:9  ] Source name: u 
[info][network_config.cpp:10 ] Target name: v 
[info][ubodt_gen_app_config.cpp:76 ] Delta 0.03
[info][ubodt_gen_app_config.cpp:77 ] Output file ../data/fmm_data/ubodt.txt
[info][ubodt_gen_app_config.cpp:78 ] Log level 2-info
[info][ubodt_gen_app_config.cpp:79 ] Use omp true
[info][ubodt_gen_app_config.cpp:80 ] ---- Print configuration done ----
[info][ubodt_gen_app_config.cpp:97 ] Validating configuration for UBODT construction
[warning][ubodt_gen_app_config.cpp:102] Overwrite result file ../data/fmm_data/ubodt.txt
[info][ubodt_gen_app_config.cpp:1

bash: line 1: fg: no job control
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
